In [28]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

class TransformerEncoder(nn.Module):
    def __init__(self, input_dim, d_model, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerEncoder, self).__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=nhid, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer=encoder_layers, num_layers=nlayers)
        self.encoder = nn.Linear(input_dim, d_model)
        # self.decoder = nn.Linear(d_model, input_dim)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        # self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        # output = self.decoder(output)
        return output


In [29]:
# 使用简化的模型参数进行示例
d_model = 512  # 模型的隐藏层维度
nhead = 8  # 多头注意力机制中的头数
nhid = 2048  # 前馈网络的维度
nlayers = 2  # Transformer编码器层的数量，为简化起见使用2层
dropout = 0.1  # Dropout率

# 实例化模型
model = TransformerEncoder(input_dim=2, d_model=d_model, nhead=nhead, nhid=nhid, nlayers=nlayers, dropout=dropout)

In [30]:
# 示例数据
times = [12, 13, 14, 16]  # 时间点（小时）
temperatures = [28, 29, 27, 20]  # 温度值

input_data = []
for t, temp in zip(times, temperatures):
    input_data.append([t, temp])
# 将数据转换为模型期望的格式 [seq_len, batch_size, input_dim]
# 这里，seq_len=4（因为有4个时间点），batch_size=1（单个序列），input_dim=1（每个时间点的温度值）
input_features = torch.tensor(input_data, dtype=torch.float).unsqueeze(1)

print(input_features.shape)  # 输出：torch.Size([4, 1, 1])

torch.Size([4, 1, 2])


In [31]:
# 注意：在实际应用中，你需要先训练模型，这里假设模型已经训练完成

# 使用模型对数据进行编码
encoded_features = model(input_features)

print(encoded_features.shape)  # 输出形状，例如：torch.Size([4, 1, 512])

torch.Size([4, 1, 512])
